In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy import stats


In [2]:
ipldata = pd.read_csv('IPL2023details.csv')
bat1 = pd.read_csv('batting1.csv')
bat2 = pd.read_csv('batting2.csv')
bowl1 = pd.read_csv('Bowling1.csv')
bowl2 = pd.read_csv('Bowling2.csv')

In [3]:
iplbat1 = ipldata[['First_innings','counter']]
iplbat2 = ipldata[['second_innings','counter']]

In [4]:
dfbat1 = pd.merge(bat1,ipldata[['First_innings','counter']],how = 'inner',on = ['counter'])
dfbat2 = pd.merge(bat2,ipldata[['second_innings','counter']],how = 'inner',on = ['counter'])
dfbowl1 = pd.merge(bowl1,ipldata[['second_innings','counter']],how = 'inner',on = ['counter'])
dfbowl2 = pd.merge(bowl2,ipldata[['First_innings','counter']],how = 'inner',on = ['counter'])
dfbat1.rename(columns={'First_innings':'bat_team'},inplace=True)
dfbat2.rename(columns={'second_innings':'bat_team'},inplace=True)
dfbowl2.rename(columns={'First_innings':'bowl_team'},inplace=True)
dfbowl1.rename(columns={'second_innings':'bowl_team'},inplace=True)
bat = pd.concat([dfbat1,dfbat2],axis=0)
bowl = pd.concat([dfbowl1,dfbowl2],axis=0)
dfbat = pd.merge(bat,ipldata,how = 'inner',on = ['counter'])
dfbowl = pd.merge(bowl,ipldata,how = 'inner',on = ['counter'])


In [6]:
dfbat.to_csv('batipldata.csv',header=dfbat.columns, index=False, encoding='utf-8' )
dfbowl.to_csv('bowipldata.csv',header=dfbowl.columns, index=False, encoding='utf-8' )

In [8]:
teambat1 = dfbat.loc[dfbat['bat_team'] == 'Punjab Kings']
teambat2 = dfbat.loc[dfbat['bat_team'] == 'Lucknow Super Giants']
teambowl1 = dfbowl.loc[dfbowl['bowl_team'] == 'Lucknow Super Giants']
teambowl2 = dfbowl.loc[dfbowl['bowl_team'] == 'Punjab Kings']
team1 = teambat1[['out','Date']]
team2 = teambat2[['out','Date']]

In [9]:
teambat1['bat_team'].iloc[1]

'Punjab Kings'

In [10]:
teambat1['counter'].value_counts().count()

7

In [11]:
total_bowl_team1 = teambowl1.groupby(['Date']).sum()[['Runs','Wicket','Zeros','fours','sixs','Wide','Noball']]
total_bowl_team2 = teambowl2.groupby(['Date']).sum()[['Runs','Wicket','Zeros','fours','sixs','Wide','Noball']]

In [12]:
total_bat_team1 = teambat1.groupby(['Date']).sum()[['runs','fours','sixs']]
total_bat_team2 = teambat2.groupby(['Date']).sum()[['runs','fours','sixs']]


In [13]:
caught = team1.loc[team1['out'].str.startswith('c ')].groupby(['Date']).count()
caught.rename(columns={'out':'caught1'},inplace=True)
runout = team1.loc[team1['out'].str.startswith('run ')].groupby(['Date']).count()
runout.rename(columns={'out':'runout1'},inplace=True)
lbw = team1.loc[team1['out'].str.startswith('lbw ')].groupby(['Date']).count()
lbw.rename(columns={'out':'lbw1'},inplace=True)
bowled = team1.loc[team1['out'].str.startswith('b ')].groupby(['Date']).count()
bowled.rename(columns={'out':'bowled1'},inplace=True)
stumping = team1.loc[team1['out'].str.startswith('st ')].groupby(['Date']).count()
stumping.rename(columns={'out':'stumping1'},inplace=True)
notout = team1.loc[team1['out'].str.startswith('not ')].groupby(['Date']).count()
notout.rename(columns={'out':'notout1'},inplace=True)
df_out_team1 = pd.concat([notout,caught,bowled,runout,lbw,stumping],axis=1).fillna(0)

In [14]:
caught = team2.loc[team2['out'].str.startswith('c ')].groupby(['Date']).count()
caught.rename(columns={'out':'caught2'},inplace=True)
runout = team2.loc[team2['out'].str.startswith('run ')].groupby(['Date']).count()
runout.rename(columns={'out':'runout2'},inplace=True)
lbw = team2.loc[team2['out'].str.startswith('lbw ')].groupby(['Date']).count()
lbw.rename(columns={'out':'lbw2'},inplace=True)
bowled = team2.loc[team2['out'].str.startswith('b ')].groupby(['Date']).count()
bowled.rename(columns={'out':'bowled2'},inplace=True)
stumping = team2.loc[team2['out'].str.startswith('st ')].groupby(['Date']).count()
stumping.rename(columns={'out':'stumping2'},inplace=True)
notout = team2.loc[team2['out'].str.startswith('not ')].groupby(['Date']).count()
notout.rename(columns={'out':'notout2'},inplace=True)
df_out_team2 = pd.concat([notout,caught,bowled,runout,lbw,stumping],axis=1).fillna(0)

In [15]:
def ciinterval(df,a):
    for col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        n = len(df[col])
        se = std / np.sqrt(n)
        ci = stats.t.interval(a, n - 1, loc=mean, scale=se)
        print(f"Column {col} has a confidence interval of {ci}")

In [16]:
# total_bowl_team1
# total_bowl_team2
# total_bat_team1
# total_bat_team2
# df_out_team1
# # df_out_team2

In [17]:
ciinterval(total_bat_team1,0.95)

Column runs has a confidence interval of (140.47306500555015, 189.24122070873557)
Column fours has a confidence interval of (12.803626743245452, 18.624944685325975)
Column sixs has a confidence interval of (5.246225525611521, 11.039488760102763)


In [18]:
ciinterval(df_out_team1,0.95)

Column notout1 has a confidence interval of (1.0770777253581973, 2.0657794174989457)
Column caught1 has a confidence interval of (1.7327801095653903, 5.410077033291753)
Column bowled1 has a confidence interval of (0.7334684136817826, 2.980817300603932)
Column runout1 has a confidence interval of (-0.1654180636228305, 1.5939894921942592)
Column lbw1 has a confidence interval of (-0.41272521458316147, 2.4127252145831615)
Column stumping1 has a confidence interval of (-0.16556423047656094, 0.7369928019051324)


In [19]:
ciinterval(total_bowl_team1,0.95)

Column Runs has a confidence interval of (125.66146091196613, 190.62425337374816)
Column Wicket has a confidence interval of (4.169030421303006, 8.402398150125565)
Column Zeros has a confidence interval of (39.53124027171255, 53.32590258543031)
Column fours has a confidence interval of (10.385388626790988, 15.328897087494727)
Column sixs has a confidence interval of (1.9469388013816404, 11.48163262718979)
Column Wide has a confidence interval of (1.3114704064021347, 4.688529593597865)
Column Noball has a confidence interval of (-0.06577941749894545, 0.9229222746418025)


In [20]:
# -------------------Second - innings--------------

In [21]:
ciinterval(total_bat_team2,0.95)

Column runs has a confidence interval of (126.388300760261, 187.89741352545326)
Column fours has a confidence interval of (8.665421235751431, 15.334578764248569)
Column sixs has a confidence interval of (2.8902975765434293, 12.252559566313714)


In [22]:
ciinterval(df_out_team2,0.95)

Column notout2 has a confidence interval of (1.5075840216011887, 2.206701692684526)
Column caught2 has a confidence interval of (3.138037621522386, 6.290533807049043)
Column bowled2 has a confidence interval of (0.24486660933901594, 1.755133390660984)
Column runout2 has a confidence interval of (-0.3311284609531219, 1.4739856038102648)
Column lbw2 has a confidence interval of (-0.29909331418863655, 1.1562361713314937)
Column stumping2 has a confidence interval of (-0.20670169268452576, 0.49241597839881146)


In [23]:
ciinterval(total_bowl_team2,0.95)

Column Runs has a confidence interval of (141.93477848295998, 185.49379294561146)
Column Wicket has a confidence interval of (3.381768140120262, 7.189660431308309)
Column Zeros has a confidence interval of (34.95787234848985, 43.32784193722444)
Column fours has a confidence interval of (11.418087822398352, 18.581912177601648)
Column sixs has a confidence interval of (3.0065617432939478, 9.564866828134623)
Column Wide has a confidence interval of (0.23626801422848498, 4.620874842914372)
Column Noball has a confidence interval of (-0.1562361713314937, 1.2990933141886365)


In [23]:
total_bat_team1.describe()

,runs,fours,sixs
count,7.000000,7.000000,7.000000
mean,173.428571,15.571429,8.142857
std,21.298334,5.094348,3.532165
min,132.000000,10.000000,4.000000
25%,168.000000,12.000000,6.500000
50%,176.000000,13.000000,7.000000
75%,187.500000,19.500000,9.000000
max,195.000000,23.000000,15.000000


In [27]:
df_out_team1.describe()

,notout1,caught1,bowled1,runout1,lbw1,stumping1
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.0
mean,1.833333,3.500000,1.000000,0.333333,0.166667,0.0
std,0.408248,1.516575,0.632456,0.816497,0.408248,0.0
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.0
25%,2.000000,3.000000,1.000000,0.000000,0.000000,0.0
50%,2.000000,3.500000,1.000000,0.000000,0.000000,0.0
75%,2.000000,4.750000,1.000000,0.000000,0.000000,0.0
max,2.000000,5.000000,2.000000,2.000000,1.000000,0.0


In [23]:
total_bowl_team1.describe()

,Runs,Wicket,Zeros,fours,sixs,Wide,Noball
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,174.500000,5.666667,41.166667,16.666667,7.333333,3.666667,0.833333
std,43.311661,2.250926,8.773065,3.011091,4.760952,2.338090,0.752773
min,118.000000,4.000000,30.000000,12.000000,0.000000,1.000000,0.000000
25%,141.000000,4.250000,35.000000,15.500000,4.750000,2.250000,0.250000
50%,188.000000,5.000000,40.500000,17.000000,8.000000,3.000000,1.000000
75%,196.750000,5.750000,48.250000,17.750000,10.500000,5.250000,1.000000
max,228.000000,10.000000,52.000000,21.000000,13.000000,7.000000,2.000000


In [133]:
print('95% confident',st.t.interval(confidence = 0.95,df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))
print('99% confident',st.t.interval(confidence = 0.99,df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))


95% confident (10.82789234874415, 15.505440984589182)
99% confident (9.4981304925549, 16.835202840778432)
